# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 67 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.16510


extracting tarball to tmp_2211.16510... done.


Retrieving document from  https://arxiv.org/e-print/2211.16516


extracting tarball to tmp_2211.16516...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.17087


extracting tarball to tmp_2211.17087...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.17097


extracting tarball to tmp_2211.17097...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16510-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16510) | **First release of PLATO consortium stellar limb-darkening coefficients**  |
|| Giuseppe Morello, et al. -- incl., <mark>Maria Bergemann</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *Published on RNAAS*|
|**Abstract**| We release the first grid of stellar limb-darkening coefficients (LDCs) and intensity profiles (IPs) computed by the consortium of the PLAnetary Transits and Oscillations of stars (PLATO), the next medium-class (M3) mission under development by the European Space Agency (ESA) to be launched in 2026. We have performed spectral synthesis with \texttt{TurboSpectrum} on a grid of \texttt{MARCS} model atmospheres. Finally, we adopted \texttt{ExoTETHyS} to convolve the high-resolution spectra ($R=2\times10^5$) with the state-of-the-art response functions for all the PLATO cameras, and computed the LDCs that best approximate the convolved IPs. In addition to the PLATO products, we provide new LDCs and IPs for the Kepler mission, based on the same grid of stellar atmospheric models and calculation procedures. The data can be downloaded from the following link: \url{https://doi.org/10.5281/zenodo.7339706}. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16996-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16996) | **High-$z$ Quasar Candidate Archive: A Spectroscopic Catalog of Quasars  and Contaminants in Various Quasar Searches**  |
|| Da-Ming Yang, et al. -- incl., <mark>Joseph F. Hennawi</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *28 pages, 26 figures, 4 tables*|
|**Abstract**| We present the high-$z$ quasar candidate archive (HzQCA), summarizing the spectroscopic observations of 174 $z\gtrsim5$ quasar candidates using Keck/LRIS, Keck/MOSFIRE, and Keck/NIRES. We identify 7 candidates as $z\sim 6$ quasars 3 of them newly reported here, and 51 candidates as brown dwarfs. In the remaining sources, 74 candidates are unlikely to be quasars; 2 sources are inconclusive; the others could not be fully reduced or extracted. Based on the classifications we investigate the distributions of quasars and contaminants in color space with photometry measurements from DELS ($z$), VIKING/UKIDSS ($YJHK_s$/$YJHK$), and un\textit{WISE} ($W1W2$). We find that the identified brown dwarfs are not fully consistent with the empirical brown dwarf model that is commonly used in quasar candidate selection methods. To refine spectroscopic confirmation strategies, we simulate synthetic spectroscopy of high-$z$ quasars and contaminants for all three instruments. The simulations utilize the spectroscopic data in HzQCA. We predict the required exposure times for quasar confirmation and propose and optimal strategy for spectroscopic follow-up observations. For example, we demonstrate that we can identify a $m_J=21.5$ at $z=7.6$ or a $m_J=23.0$ at $z=7.0$ within 15\,min of exposure time with LRIS. With the publication of the HzQCA we aim to provide guidance for future quasar surveys and candidate classification. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16516-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16516) | **Weak Lensing Tomographic Redshift Distribution Inference for the Hyper  Suprime-Cam Subaru Strategic Program three-year shape catalogue**  |
|| Markus Michael Rau, et al. -- incl., <mark>Tianqing Zhang</mark>, <mark>Xiangchong Li</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *23 pages, 11 figures, 1 table, submitted to the MNRAS; comments welcome*|
|**Abstract**| We present posterior sample redshift distributions for the Hyper Suprime-Cam Subaru Strategic Program Weak Lensing three-year (HSC Y3) analysis. Using the galaxies' photometry and spatial cross-correlations, we conduct a combined Bayesian Hierarchical Inference of the sample redshift distributions. The spatial cross-correlations are derived using a subsample of Luminous Red Galaxies (LRGs) with accurate redshift information available up to a photometric redshift of $z < 1.2$. We derive the photometry-based constraints using a combination of two empirical techniques calibrated on spectroscopic- and multiband photometric data that covers a spatial subset of the shear catalog. The limited spatial coverage induces a cosmic variance error budget that we include in the inference. Our cross-correlation analysis models the photometric redshift error of the LRGs to correct for systematic biases and statistical uncertainties. We demonstrate consistency between the sample redshift distributions derived using the spatial cross-correlations, the photometry, and the posterior of the combined analysis. Based on this assessment, we recommend conservative priors for sample redshift distributions of tomographic bins used in the three-year cosmological Weak Lensing analyses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.17087-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.17087) | **Precession of magnetars: dynamical evolutions and modulations on  polarized electromagnetic waves**  |
|| Yong Gao, et al. -- incl., <mark>Lijing Shao</mark>, <mark>Michael Kramer</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *18 pages, 18 figures; accepted by MNRAS*|
|**Abstract**| Magnetars are conjectured to be highly magnetized neutron stars (NSs). Strong internal magnetic field and elasticity in the crust may deform the stars and lead to free precession. We study the precession dynamics of triaxially-deformed NSs incorporating the near-field and the far-field electromagnetic torques. We obtain timing residuals for different NS geometries and torques. We also investigate the polarized X-ray and radio signals from precessing magnetars. The modulations on the Stokes parameters are obtained for thermal X-rays emitted from the surface of magnetars. For radio signals, we apply the simple rotating vector model (RVM) to give the modulations on the position angle (PA) of the polarization. Our results are comprehensive, ready to be used to search for magnetar precession with timing data and polarizations of X-ray and radio emissions. Future observations of precessing magnetars will give us valuable information on the geometry and the strength of the strong magnetic fields, the emission geometry, as well as the equation of state (EoS) of NSs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.17097-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.17097) | **The rise and fall of the iron-strong nuclear transient PS16dtm**  |
|| T. Petrushevska, et al. -- incl., <mark>K. W. Smith</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *Accepted for publication in A&A, 19 pages and 18 figures*|
|**Abstract**| Thanks to the advent of large-scale optical surveys, a diverse set of flares from the nuclear regions of galaxies has recently been discovered. These include the disruption of stars by supermassive black holes at the centers of galaxies - nuclear transients known as tidal disruption events (TDEs). Active galactic nuclei (AGN) can show extreme changes in the brightness and emission line intensities, often referred to as changing-look AGN (CLAGN). Given the physical and observational similarities, the interpretation and distinction of nuclear transients as CLAGN or TDEs remains difficult. One of the obstacles of making progress in the field is the lack of well-sampled data of long-lived nuclear outbursts in AGN. Here, we study PS16dtm, a nuclear transient in a Narrow Line Seyfert 1 (NLSy1) galaxy, which has been proposed to be a TDE candidate. Our aim is to study the spectroscopic and photometric properties of PS16dtm, in order to better understand the outbursts originating in NLSy1 galaxies. Our extensive multiwavelength follow-up that spans around 2000 days includes photometry and spectroscopy in the UV/optical, as well as mid-infrared (MIR) and X-ray observations. Furthermore, we improved an existing semiempirical model in order to reproduce the spectra and study the evolution of the spectral lines. The UV/optical light curve shows a double peak at $\sim50$ and $\sim100$ days after the first detection, and it declines and flattens afterward, reaching preoutburst levels after 2000 days of monitoring. The MIR light curve rises almost simultaneously with the optical, but unlike the UV/optical which is approaching the preoutburst levels in the last epochs of our observations, the MIR emission is still rising at the time of writing. The optical spectra show broad Balmer features and the strongest broad Fe II emission ever detected in a nuclear transient. [abridged] |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16979-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16979) | **Using simulated Tianqin gravitational wave data and electromagnetic wave  data to study the coincidence problem and Hubble tension problem**  |
|| <mark>JiaWei Zhang</mark>, et al. -- incl., <mark>Jin Li</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *The article has been accepted by Chinese Physics C*|
|**Abstract**| In this paper, we use electromagnetic wave data (H0LiCOW, $H(z)$, SNe) and gravitational wave data (Tianqin) to constrain the interacting dark energy (IDE) model and investigate the Hubble tension problem and coincidences problem. By combining these four kinds of data (Tianqin+H0LiCOW+SNe+$H(z)$), we obtained the parameter values at the confidence interval of $1\sigma$: $\Omega_m=0.36\pm0.18$, $\omega_x=-1.29^{+0.61}_{-0.23}$, $\xi=3.15^{+0.36}_{-1.1}$, and $H_0=70.04\pm0.42$ $kms^{-1}Mpc^{-1}$. According to our results, the best valve of $H_0$ show that the Hubble tension problem can be alleviated to some extent. In addition, the $\xi+3\omega_x = -0.72^{+2.19}_{-1.19}(1\sigma)$ of which the center value indicates the coincidence problem is slightly alleviated. However, the $\xi+3\omega_x = 0$ is still within the $1\sigma$ error range which indicates the $\Lambda$CDM model is still the model which is in best agreement with the observational data at present. Finally, we compare the constraint results of electromagnetic wave and gravitational wave on the model parameters and find that the constraint effect of electromagnetic wave data on model parameters is better than that of simulated Tianqin gravitational wave data. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xc0 in position 35572: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16748-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16748) | **Extreme nature of four blue-excess dust-obscured galaxies revealed by  optical spectroscopy**  |
|| Akatoki Noboriguchi, et al. -- incl., <mark>Yoshihiro Ueda</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *22 pages, 12 figures, accepted for publication in ApJ*|
|**Abstract**| We report optical spectroscopic observations of four blue-excess dust-obscured galaxies (BluDOGs) identified by Subaru Hyper Suprime-Cam. BluDOGs are a sub-class of dust-obscured galaxies (DOGs, defined with the extremely red color $(i-[22])_{\rm AB} \geq 7.0$; Toba et al. 2015), showing a significant flux excess in the optical $g$- and $r$-bands over the power-law fits to the fluxes at the longer wavelengths. Noboriguchi et al. (2019) has suggested that BluDOGs may correspond to the blowing-out phase involved in a gas-rich major merger scenario. However the detailed properties of BluDOGs are not understood because of the lack of spectroscopic information. In this work, we carry out deep optical spectroscopic observations of four BluDOGs using Subaru/FOCAS and VLT/FORS2. The obtained spectra show broad emission lines with extremely large equivalent widths, and a blue wing in the CIV line profile. The redshifts are between 2.2 and 3.3. The averaged rest-frame equivalent widths of the CIV lines are $160\pm33$ $\mathrm{\mathring{A}}$, $\sim$7 times higher than the average of a typical type-1 quasar. The FWHMs of their velocity profiles are between 1990 and 4470 ${\rm km\ s^{-1}}$, and their asymmetric parameters are 0.05 and 0.25. Such strong CIV lines significantly affect the broad-band magnitudes, which is partly the origin of the blue excess seen in the spectral energy distribution of BluDOGs. Their estimated supermassive black hole masses are $1.1\times10^8 < M_{\rm BH}/M_\odot < 5.5 \times 10^8$. The inferred Eddington ratios of the BluDOGs are higher than 1 ($1.1< \lambda_{\rm Edd} < 3.8$), suggesting that the BluDOGs are in a rapidly evolving phase of supermassive black holes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16795-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16795) | **The MUSE second-generation VLT instrument**  |
|| Bacon R., et al. -- incl., <mark>Lilly S.J.</mark>, <mark>Lizon J.-L.</mark> |
|*Appeared on*| *2022-12-01*|
|*Comments*| *10 pages, 5 figures*|
|**Abstract**| The Multi Unit Spectroscopic Explorer (MUSE) is a second-generation VLT panoramic integral-field spectrograph currently in manufacturing, assembly and integration phase. MUSE has a field of 1x1 arcmin2 sampled at 0.2x0.2 arcsec2 and is assisted by the VLT ground layer adaptive optics ESO facility using four laser guide stars. The instrument is a large assembly of 24 identical high performance integral field units, each one composed of an advanced image slicer, a spectrograph and a 4kx4k detector. In this paper we review the progress of the manufacturing and report the performance achieved with the first integral field unit. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.16510.md
exported in  _build/html/2211.16996.md
    + _build/html/tmp_2211.16996/./figure/flux_ratio_corner_meancov.png
    + _build/html/tmp_2211.16996/./figure/throughput.png
    + _build/html/tmp_2211.16996/./figure/flux_ratio_zW1.png
    + _build/html/tmp_2211.16996/./figure/cc_zW1.png
    + _build/html/tmp_2211.16996/./figure/flux_ratio_zY.png
    + _build/html/tmp_2211.16996/./figure/cc_zY.png
    + _build/html/tmp_2211.16996/./figure/flux_ratio_YW1.png
    + _build/html/tmp_2211.16996/./figure/cc_YW1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$</div>



<div id="title">

# First release of PLATO consortium stellar limb-darkening coefficients

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.16510-b31b1b.svg)](https://arxiv.org/abs/2211.16510)<mark>Appeared on: 2022-12-01</mark> - _Published on RNAAS_

</div>
<div id="authors">

Giuseppe Morello, et al. -- incl., <mark>Maria Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** We release the first grid of stellar limb-darkening coefficients (LDCs) and intensity profiles (IPs) computed by the consortium of the PLAnetary Transits and Oscillations of stars (PLATO), the next medium-class (M3) mission under development by the European Space Agency (ESA) to be launched in 2026. We have performed spectral synthesis with\texttt{TurboSpectrum}on a grid of\texttt{MARCS}model atmospheres. Finally, we adopted\texttt{ExoTETHyS}to convolve the high-resolution spectra ($R=2\times10^5$) with the state-of-the-art response functions for all the PLATO cameras, and computed the LDCs that best approximate the convolved IPs. In addition to the PLATO products, we provide new LDCs and IPs for the Kepler mission, based on the same grid of stellar atmospheric models and calculation procedures. The data can be downloaded from the following link:\url{https://doi.org/10.5281/zenodo.7339706}.

</div>



<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\angstrom}{\textup{\AA}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\angstrom$}{\textup{\AA}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  Quasar Candidate Archive: A Spectroscopic Catalog of Quasars and Contaminants in Various Quasar Searches

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.16996-b31b1b.svg)](https://arxiv.org/abs/2211.16996)<mark>Appeared on: 2022-12-01</mark> - _28 pages, 26 figures, 4 tables_

</div>
<div id="authors">

Da-Ming Yang, et al. -- incl., <mark><mark>Joseph F. Hennawi</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present the high-$z$quasar candidate archive (HzQCA), summarizing the spectroscopic observations of 174$z\gtrsim5$quasar candidates using Keck/LRIS, Keck/MOSFIRE, and Keck/NIRES. We identify 7 candidates as$z\sim 6$quasars 3 of them newly reported here, and 51 candidates as brown dwarfs. In the remaining sources, 74 candidates are unlikely to be quasars; 2 sources are inconclusive; the others could not be fully reduced or extracted. Based on the classifications we investigate the distributions of quasars and contaminants in color space with photometry measurements from DELS ($z$), VIKING/UKIDSS ($YJHK_s$/$YJHK$), and un\textit{WISE}($W1W2$). We find that the identified brown dwarfs are not fully consistent with the empirical brown dwarf model that is commonly used in quasar candidate selection methods. To refine spectroscopic confirmation strategies, we simulate synthetic spectroscopy of high-$z$quasars and contaminants for all three instruments. The simulations utilize the spectroscopic data in HzQCA. We predict the required exposure times for quasar confirmation and propose and optimal strategy for spectroscopic follow-up observations. For example, we demonstrate that we can identify a$m_J=21.5$at$z=7.6$or a$m_J=23.0$at$z=7.0$within 15 min of exposure time with LRIS. With the publication of the HzQCA we aim to provide guidance for future quasar surveys and candidate classification.

</div>

<div id="div_fig1">

<img src="tmp_2211.16996/./figure/flux_ratio_corner_meancov.png" alt="Fig9" width="100%"/>

**Figure 9. -** The complete relative flux diagrams. The filled symbols are the sources in the archive, including QSO (pink), STAR (blue) and UNQ (brown) types objects. The red contour is the density of 440,000 simulated $6\leq z\leq 8$ quasars generated with \texttt{simqso}. The cyan contour is the density of 100,000 synthetic brown dwarfs, generated from convolving the empirical brown dwarf colors in \protect\cite{Skrzypek2015} and the measurement errors, following the noise modeling procedures described in \protect\cite{Nanni2022}. The grey contour is the density of the contaminants used to train the contaminant model in XDHZQSO searches (see Section \ref{sec:VIK+UKI}). The levels of the contours are 11.75\%, 39.35\%, 67.53\%, and 86.47\%. (*fig:flux_ratio_corner*)

</div>
<div id="div_fig2">

<img src="tmp_2211.16996/./figure/throughput.png" alt="Fig1" width="100%"/>

**Figure 1. -** The solid lines are throughput curves of the three spectrometers, corresponding to an 0$\farcs$9 seeing for LRIS, MOSFIRE, and order 7 of NIRES. The throughput of LRIS was originally measured with an 1$\arcsec$ slit width in LRIS-2203 run, with a seeing of 0$\farcs$74. The MOSFIRE Y-band throughput was obtained with a 1$\arcsec$ slit width and a 0$\farcs$76 seeing in MOSFIRE-2201 run. The NIRES throughput here was measured with a 0$\farcs$55 slit width in NIRES-1905 run with a seeing of 0$\farcs$9 in the order 7. The LRIS and MOSFIRE throughput curves are then scaled to a seeing of 0$\farcs$9. The dotted lines are throughput curves corrected for the slit loss, assuming a perfect centering of the star on the slit. The upper axis shows the corresponding Ly$\alpha$($1216$$\angstrom$) redshift of high-$z$ quasars that they can reach. For all instruments the throughput curves are determined from observation of the standard star GD153. (*fig:throughput*)

</div>
<div id="div_fig3">

<img src="tmp_2211.16996/./figure/flux_ratio_zW1.png" alt="Fig8.1" width="16%"/><img src="tmp_2211.16996/./figure/cc_zW1.png" alt="Fig8.2" width="16%"/><img src="tmp_2211.16996/./figure/flux_ratio_zY.png" alt="Fig8.3" width="16%"/><img src="tmp_2211.16996/./figure/cc_zY.png" alt="Fig8.4" width="16%"/><img src="tmp_2211.16996/./figure/flux_ratio_YW1.png" alt="Fig8.5" width="16%"/><img src="tmp_2211.16996/./figure/cc_YW1.png" alt="Fig8.6" width="16%"/>

**Figure 8. -** (Left panels) The relative flux diagrams. The filled symbols are the sources in the HzQCA, including QSO (pink), STAR (blue) and UNQ (brown) types objects. The red contour is the density of 440,000 simulated $6\leq z\leq 8$ quasars generated with \texttt{simqso}. The cyan contour is the density of 100,000 synthetic brown dwarfs, generated from convolving the empirical brown dwarf colors in \protect\cite{Skrzypek2015} and the measurement errors, following the noise modeling procedures described in \protect\cite{Nanni2022}. The cyan line in it indicates different spectral types. The green end of this track is M5 type, while the blue end is the T8 type. The grey contour is the density of the contaminants used to train the contaminant model in XDHZQSO searches (see Section \ref{sec:VIK+UKI}). The dashed lines in the upper panels are the color selection cuts from \protect\cite{Wang2017}. (Right panels) Corresponding color-color diagrams. The levels of the contours are 11.75\%, 39.35\%, 67.53\%, and 86.47\%. (*fig:flux_ratio*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

114  publications in the last 7 days.
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers